In [1]:
"""THIS TESTER IS USING THE FOLLOWING:

Dataset: 
Music21 corpus 'ryansMammoth'

Testing features:
1) Notes frequency feature vector, ngrams = range of 1-2.

Classifier: 
Logistic Regression

"""

###############################################

import music21
import pandas as pd
import numpy as np
# from pandas.tools.plotting import scatter_matrix
# import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import cross_validation
from sklearn import metrics

In [2]:
def score_generator(scores):

    num_scores = len(filepaths)
    n = 0

    while n < num_scores:
        yield music21.corpus.parse(filepaths[n])
        n += 1

In [3]:
def part_generator(score):
    
    parts = [part for part in score.parts]
    num_parts = len(parts)
    n = 0

    while n < num_parts:
        yield n
        n += 1

In [49]:
def build_feature_corpus(score, n):
    """Takes MIDI files and creates feature arrays to be used for training."""
            
    # Convert MIDI file to score and iterate over notes in score
    # Save relevant individual Note attributes into list, including Note obj (used to calc interval) and name
    score = music21.corpus.parse(score)
    all_notes = ""

    for note in music21.alpha.theoryAnalysis.theoryAnalyzer.getNotes(score, n):
        if note == None:
            pass
        else:
            all_notes += note.name + " "

    mode_at_measure_0 = music21.alpha.theoryAnalysis.theoryAnalyzer.getKeyAtMeasure(score, 0).mode
    outcome = (mode_at_measure_0 == 'major')

    print "Done"
    return all_notes, outcome

In [ ]:
def build_pipeline():

    feature_vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words=None, ngram_range=(1, 2), token_pattern=r'\w#?-?')
    classifier = LogisticRegression()
    
    return feature_vectorizer, classifier

In [ ]:
def train_classifier(feature_vectorizer, classifier, features, outcomes):
    
    cv = cross_validation.StratifiedKFold(outcomes, 5)
    feature_vectors = feature_vectorizer.fit_transform(features)
    outcomes = np.ravel(outcomes)
    
    # Running into an issue here splitting dataset bc features is not a vectorized object
    # yet, still a dictionary. Might need to keep vectorizer and classifier separated.
    precision, recall = [], []

    for training_data, testing_data in cv:
        X_train = feature_vectors[training_data]
        X_test = feature_vectors[testing_data]
        y_train = outcomes[training_data]
        y_test = outcomes[testing_data]
        
        classifier.fit(X_train, y_train)
        y_predicted = classifier.predict(X_test)
        p,r,_,_ = metrics.precision_recall_fscore_support(y_test, y_predicted)
        precision.append(p[1])
        recall.append(r[1])
    
    return classifier, precision, recall

In [35]:
# def build_feature_vector_and_fit_model(training_corpus, outcomes):

#     vectorizer = TfidfVectorizer(min_df=1, analyzer='word', stop_words=None, ngram_range=(1,2), token_pattern=r'\w#?-?')
    
#     X_train = vectorizer.fit_transform(training_corpus)
#     X_train = X_train.toarray()
#     y_train = np.ravel(outcomes)

#     classifier_LR = LogisticRegression()
#     classifier_LR.fit(X_train, y_train)

#     print "TRAINING SET SCORE: ", classifier_LR.score(X_train, y_train)

#     return vectorizer, classifier_LR

In [47]:
def predict(vectorizer, classifier, validation_corpus, outcomes):
    """Takes notes_corpus as a list of test scores (each a string of notes)."""
    
    X_test = vectorizer.transform(validation_corpus)
    print type(X_test), X_test.shape
    X_test = X_test.toarray()

    prediction = classifier.predict(X_test)
    prediction = list(prediction)
    print 'PREDICTION:', prediction
    print ""
    print 'ACTUAL OUTCOMES: ', outcomes
    
    count = 0
    for i in range(len(prediction)):
        if prediction[i] == outcomes[i]:
            count +=1
        
    print '{} correct predictions out of {} sample test files'.format(count, len(outcomes))
    print float(count)/len(outcomes) * 100

In [30]:
# ------------------------------Executable Code --------------------------------

# Read test files and construct feature and outcome datasets
scores_celtic = music21.corpus.getComposer('ryansMammoth')
scores_classical = music21.corpus.getComposer('bach') 
scores = scores_celtic + scores_classical


In [ ]:
notes_corpus = []
outcomes = []

for score in (score_generator(scores)):
    print "Beginning new score: ", score
    for n in (part_generator(score)):            
        features, outcome = build_feature_corpus(score, n)
        notes_corpus.append(features)
        outcomes.append(outcome)

In [ ]:
feature_vectorizer, classifier = build_pipeline()
classifier, precision, recall = train_classifier(feature_vectorizer, classifier, notes_corpus, outcomes)

In [33]:
# Build training features from input midi files
# training_corpus, training_outcomes = build_feature_corpus(training_files)

Done


In [36]:
vectorizer, classifier = build_feature_vector_and_fit_model(training_corpus, training_outcomes)

TRAINING SET SCORE:  0.854716981132


In [39]:
# Build features for validation data set of midi files.
validation_corpus, validation_outcomes = build_feature_corpus(validation_files)

Done


In [48]:
predict(vectorizer, classifier, validation_corpus, validation_outcomes)

<class 'scipy.sparse.csr.csr_matrix'> (529, 192)
PREDICTION: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True